In [1]:
import xspec
import batanalysis as ba
from swifttools.swift_too import GUANO, ObsQuery, Clock
import swiftbat

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from pathlib import Path
from astropy.io import fits
from astropy.time import Time, TimeDelta
import astropy.units as u
from astropy.coordinates import SkyCoord
import datetime
import os

In [2]:
tmp_download_dir="/tmp/batdata/download_examples"

ba.datadir(tmp_download_dir, mkdir=True)


PosixPath('/private/tmp/batdata/download_examples')

In [3]:
query = ObsQuery(targetid=147478)
query

2005-07-24 12:34:18,2005-07-24 13:13:12,GRB050724,00147478000,2270,64
2005-07-24 14:04:29,2005-07-24 14:49:29,GRB050724,00147478000,2561,139
2005-07-24 15:40:54,2005-07-24 16:25:54,GRB050724,00147478000,2561,139
2005-07-24 17:17:19,2005-07-24 18:02:19,GRB050724,00147478000,2572,128
2005-07-24 18:53:45,2005-07-24 19:38:45,GRB050724,00147478000,2573,127
2005-07-24 20:31:03,2005-07-24 21:15:20,GRB050724,00147478000,2530,127
2005-07-24 22:06:36,2005-07-24 22:51:36,GRB050724,00147478000,2583,117
2005-07-24 23:43:02,2005-07-25 00:28:02,GRB050724,00147478000,2583,117
2005-07-25 01:19:27,2005-07-25 02:04:27,GRB050724,00147478000,2636,64
2005-07-25 02:55:53,2005-07-25 03:40:53,GRB050724,00147478000,2636,64
2005-07-25 04:32:19,2005-07-25 05:17:19,GRB050724,00147478000,2636,64


In [4]:
query = ObsQuery(name="GRB050724")
query

2005-07-24 12:34:18,2005-07-24 13:13:12,GRB050724,00147478000,2270,64
2005-07-24 14:04:29,2005-07-24 14:49:29,GRB050724,00147478000,2561,139
2005-07-24 15:40:54,2005-07-24 16:25:54,GRB050724,00147478000,2561,139
2005-07-24 17:17:19,2005-07-24 18:02:19,GRB050724,00147478000,2572,128
2005-07-24 18:53:45,2005-07-24 19:38:45,GRB050724,00147478000,2573,127
2005-07-24 20:31:03,2005-07-24 21:15:20,GRB050724,00147478000,2530,127
2005-07-24 22:06:36,2005-07-24 22:51:36,GRB050724,00147478000,2583,117
2005-07-24 23:43:02,2005-07-25 00:28:02,GRB050724,00147478000,2583,117
2005-07-25 01:19:27,2005-07-25 02:04:27,GRB050724,00147478000,2636,64
2005-07-25 02:55:53,2005-07-25 03:40:53,GRB050724,00147478000,2636,64
2005-07-25 04:32:19,2005-07-25 05:17:19,GRB050724,00147478000,2636,64


In [5]:
download = ba.download_swiftdata(observations=query[0].obsid)
print(download)

{'00147478000': {'obsid': '00147478000', 'success': True, 'obsoutdir': PosixPath('/private/tmp/batdata/download_examples/00147478000'), 'quicklook': False, 'data': Swift_Data(username='anonymous',obsid='00147478000',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False',tdrss='True')}}


/Users/tparsota/opt/anaconda3/envs/py311/lib/python3.11/site-packages/swifttools/swift_too/swift_data.py:405: UserWarning: SWIFT_TLE_ARCHIVE.txt.14335.67560079.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/opt/anaconda3/envs/py311/lib/python3.11/site-packages/swifttools/swift_too/swift_data.py:405: UserWarning: sw00147478000pat.fits.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/opt/anaconda3/envs/py311/lib/python3.11/site-packages/swifttools/swift_too/swift_data.py:405: UserWarning: sw00147478000pjb.par.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/opt/anaconda3/envs/py311/lib/python3.11/site-packages/swifttools/swift_too/swift_data.py:405: UserWarning: sw00147478000pob.cat.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/opt/anaconda3/envs/py311/lib/python3.11/site-packages/swifttools/swift_too/swift_data.py:405: UserWarning: sw00147478000ppr

In [6]:
event=ba.BatEvent(query[0].obsid)

RuntimeError: There are duplicate events in the merged event file /private/tmp/batdata/download_examples/00147478000/bat/event/total_events.evt.

In [ ]:
lc=event.create_lightcurve()
fig, ax =lc.plot(T0=event.trigtime_met, plot_relative=True)
ax.set_xlim([-1, 1])

In [ ]:
lc.set_timebins(timebinalg="bayesian", save_durations=True)

In [ ]:
print(f"The measured T90 is {lc.tdurs['T90']['TSTOP']-lc.tdurs['T90']['TSTART']}\nmeasured from {lc.tdurs['T90']['TSTART']-event.trigtime_met*u.s}-{lc.tdurs['T90']['TSTOP']-event.trigtime_met*u.s}" )
print(f"The measured {lc.tdurs['PEAK']['TSTOP']-lc.tdurs['PEAK']['TSTART']} peak is \nmeasured from {lc.tdurs['PEAK']['TSTART']-event.trigtime_met*u.s}-{lc.tdurs['PEAK']['TSTOP']-event.trigtime_met*u.s}" )
print(f"The measured T100 is {lc.tdurs['TOT']['TSTOP']-lc.tdurs['TOT']['TSTART']}\nmeasured from {lc.tdurs['TOT']['TSTART']-event.trigtime_met*u.s}-{lc.tdurs['TOT']['TSTOP']-event.trigtime_met*u.s}" )

In [ ]:
fig, ax = lc.plot(T0=event.trigtime_met, plot_relative=True)
ax.set_xlim([-5, +120])
print(lc.tdurs)
print(lc.tbins)

In [ ]:
spectrum_t90=event.create_pha(tstart=lc.tdurs['T90']['TSTART'], tstop=lc.tdurs['T90']['TSTOP'])

In [ ]:
ba.fit_spectrum(spectrum_t90)

In [ ]:
spectrum_t90.spectral_model

In [ ]:
bayesian_spectra=event.create_pha(tstart=lc.tbins["TIME_START"], tstop=lc.tbins["TIME_STOP"])
output_bayesian_spectra=ba.parallel.batspectrum_analysis(bayesian_spectra, nprocs=1)

In [ ]:
ba.plot_TTE_lightcurve(lc, [spectrum_t90], plot_relative=True, T0=event.trigtime_met)
ba.plot_TTE_lightcurve(lc, output_bayesian_spectra, plot_relative=True, T0=event.trigtime_met)


In [ ]:
lc_1s=event.create_lightcurve(timebinalg="uniform", timedelta=np.timedelta64(1, "s"))#, tstart=-1*u.s, tstop=1*u.s, is_relative=True, T0=event.trigtime_met)
fig, ax =lc_1s.plot(T0=event.trigtime_met, plot_relative=True)

In [ ]:
lc_1s.set_timebins(tmin=-10*u.s, tmax=150*u.s, timedelta=np.timedelta64(1, "s"), is_relative=True, T0=event.trigtime_met)
fig, ax =lc_1s.plot(energybins=[15,350]*u.keV, T0=event.trigtime_met, plot_relative=True)
ax.axhline(0, c='k', ls='--')

ax.axvspan((lc.tdurs['T90']['TSTART']-event.trigtime_met*u.s).value, (lc.tdurs['T90']['TSTOP']-event.trigtime_met*u.s).value, color='g', alpha=0.3, label="T90")
ax.legend()


In [ ]:
spectrum_peak_1s=event.create_pha(tstart=lc.tdurs['PEAK']['TSTART'], tstop=lc.tdurs['PEAK']['TSTOP'])
spectrum_peak_1s.plot()

In [ ]:
ba.fit_spectrum(spectrum_peak_1s, use_cstat=False)

In [ ]:
peak_flux = event.create_lightcurve(tstart=lc.tdurs['PEAK']['TSTART'],tstop=lc.tdurs['PEAK']['TSTOP'], timedelta=np.timedelta64(0, "s"))
print(peak_flux.data["RATE"]*(lc.tdurs['PEAK']['TSTOP']-lc.tdurs['PEAK']['TSTART']))


In [ ]:
spectrum_peak_tot=event.create_pha(tstart=lc.tdurs['TOT']['TSTART'], tstop=lc.tdurs['TOT']['TSTOP'])
ba.fit_spectrum(spectrum_peak_tot, use_cstat=False)


In [ ]:
total_fluence = event.create_lightcurve(tstart=lc.tdurs['TOT']['TSTART'], tstop=lc.tdurs['TOT']['TSTOP'], timedelta=np.timedelta64(0, "s"))
print(total_fluence.data["RATE"]*(lc.tdurs['TOT']['TSTOP']-lc.tdurs['TOT']['TSTART']))

In [ ]:
ba.fit_spectrum(spectrum_peak_tot, use_cstat=False, generic_model="cflux*grbm", setPars={1:"15,-1", 2:"150,-1",3:-8, 4:"-2",5:"-2",6:100, 7:"1e-3,-1"}, fit_iterations=10000)

In [ ]:
ba.fit_TTE_spectrum?

In [ ]:
xspec.Model("cflux*grbm", )